# This example demonstrate how to train YOLO detector model

In [1]:
import sys
import os
import json
import yaml
import copy
import glob
import shutil
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../../../../')
sys.path.append(NOMEROFF_NET_DIR)

from nomeroff_net.tools.yolo_tools import convert_dataset_to_yolo_format

In [2]:
plt.rcParams["figure.figsize"] = (20, 10)

In [3]:
%matplotlib inline 

# Download dataset and yolo repo

In [4]:
# auto download latest dataset
from nomeroff_net.tools import modelhub

# get yolov5
info = modelhub.download_repo_for_model("yolox")
repo_path = info["repo_path"]

# # auto download latest dataset
info = modelhub.download_dataset_for_model("yolox")
PATH_TO_DATASET = info["dataset_path"]

# local path dataset
#PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR, 
#                               "./data/dataset/Detector/autoria_numberplate_dataset_example")

In [5]:
sys.path.append(repo_path)

In [6]:
PATH_TO_DATASET

'/mnt/raid/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/Detector/yolox/autoriaNumberplateDataset-2021-12-12'

# Convert dataset to COCO format

In [7]:
# init coco blocks
coco = {
    "info": {},
    "licenses": [],
    "images": [],
    "annotations": [],
    "categories": [],
    "segment_info": []
}

# init info block
coco["info"] = {
    "description": "NumberPlate Dataset",
    "url": "https://nomeroff.net.ua",
    "version": "0.3.1",
    "year": 2019,
    "contributor": "RIA.com Marketplaces",
    "date_created": "2019/09/17"
}

# init licenses block
coco["licenses"] = [
    {
        "url": "https://github.com/ria-com/nomeroff-net/blob/master/LICENSE",
        "id": 1,
        "name": "GNU General Public License v3.0"
    }
]

# init categories
coco["categories"] = [
    {"supercategory": "vehicle","id": 1,"name": "numberplate"}
]

In [8]:
# create own coco datasets from VIA VGG
ROOT_DIR = PATH_TO_DATASET
DATASETS = ["train", "val"]
VIA_JSON_NAMES = ["train/via_region_data.json", "val/via_region_data.json"]

ALLOWED_FORMATS = ["png", "jpg", "jpeg"]

# init default data
coco_datasets = {}
for dataset in DATASETS:
    coco_datasets[dataset] = copy.deepcopy(coco)

In [9]:
# load images
for dataset in DATASETS:
    for _id, img_path in enumerate(glob.glob(f"{os.path.join(ROOT_DIR, dataset)}/*")):
        if os.path.basename(img_path).split(".")[-1].lower() not in ALLOWED_FORMATS:
            continue
        
        # copy to res image path
        #shutil.copyfile(img_path, os.path.join(ANN_RES_DIR, os.path.basename(img_path)))
        
        # get image metadata
        image = Image.open(img_path)
        exif = image.getexif()
        creation_time = exif.get(36867)
        width, height = image.size
        
        # append in coco dataset format
        coco_datasets[dataset]["images"].append({
            "license": 1,
            "file_name": os.path.basename(img_path),
            "coco_url": "https://nomeroff.net.ua/datasets/autoriaNumberplateDataset-2019-09-17.zip",
            "height": height,
            "width":  width,
            "date_captured": creation_time,
            "flickr_url": "http://farm7.staticflickr.com/6116/6255196340_da26cf2c9e_z.jpg",
            "id": _id
        })

In [10]:
def PolygonSort(corners):
    n = len(corners)
    cx = float(sum(x for x, y in corners)) / n
    cy = float(sum(y for x, y in corners)) / n
    cornersWithAngles = []
    for x, y in corners:
        an = (np.arctan2(y - cy, x - cx) + 2.0 * np.pi) % (2.0 * np.pi)
        cornersWithAngles.append((x, y, an))
    cornersWithAngles.sort(key = lambda tup: tup[2])
    return  [(x, y) for x, y, an in cornersWithAngles]

def PolygonArea(corners):
    n = len(corners)
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += corners[i][0] * corners[j][1]
        area -= corners[j][0] * corners[i][1]
    area = abs(area) / 2.0
    return area

In [11]:
# add annotations
annotation_id = 0
for json_data_path, dataset in zip(VIA_JSON_NAMES, DATASETS):
    with open(os.path.join(ROOT_DIR, json_data_path)) as jsonFile:
        json_data = json.load(jsonFile)
    for key in json_data["_via_img_metadata"]:
        metadata = json_data["_via_img_metadata"][key]
        
        # define image_id
        image_filename = metadata["filename"]
        image_id = [x["id"] for x in coco_datasets[dataset]["images"] if x["file_name"] == image_filename][0]
            
        for region in metadata["regions"]:
            if region["shape_attributes"].get("all_points_x", None) is None \
                or region["shape_attributes"].get("all_points_y", None) is None:
                    print(f"[FAIL] can not read all_points_x/y in {image_filename}")
                    continue
            segmentation = [[]]
            for x, y in zip(region["shape_attributes"]["all_points_x"], region["shape_attributes"]["all_points_y"]):
                segmentation[0].append(x)
                segmentation[0].append(y)
            
            # define area
            corners = [(x, y) for x, y in zip(region["shape_attributes"]["all_points_x"], region["shape_attributes"]["all_points_y"])]
            corners_sorted = PolygonSort(corners)
            area = PolygonArea(corners_sorted)
            
            # define category_id
            category_name = region["region_attributes"].get("class", 1)
            #category_ids = [x["id"] for x in coco_datasets[dataset]["categories"] if x["name"] == category_name]
            
            #if not len(category_ids):
            #    continue
            category_id = 1#category_ids[0]
            # define bbox
            x_min = min([c[0] for c in corners])
            x_max = max([c[0] for c in corners])
            y_min = min([c[1] for c in corners])
            y_max = max([c[1] for c in corners])
            
            
            # appent annotation
            coco_datasets[dataset]["annotations"].append({
                "segmentation": segmentation,
                "area": area,
                "iscrowd": 0,
                "image_id": image_id,
                "bbox": [x_min, y_min, x_max-x_min, y_max-y_min],
                "category_id": category_id,
                "id": annotation_id
            })
            annotation_id += 1

[FAIL] can not read all_points_x/y in 369346652-1.jpeg
[FAIL] can not read all_points_x/y in 369349937-28729111.jpeg
[FAIL] can not read all_points_x/y in 369355416-28729416.jpeg


In [12]:
# save datasets json
for dataset in DATASETS:
    res_dir = os.path.join(ROOT_DIR, "annotations")
    os.makedirs(res_dir, exist_ok=True)
    with open(os.path.join(res_dir, f"yolox_{dataset}.json"), "w") as jsonFile:
        json.dump(coco_datasets[dataset], jsonFile)

# Crete yolox dataset class

In [13]:
ROOT_DIR

'/mnt/raid/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/Detector/yolox/autoriaNumberplateDataset-2021-12-12'

In [32]:
%%writefile {NOMEROFF_NET_DIR}/data/configs/yolox_s.py
import os
import torch
from yolox.exp import Exp as MyExp


class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        # yolox_s
        self.depth = 0.33
        self.width = 0.50
        self.exp_name = os.path.split(os.path.realpath(__file__))[1].split(".")[0]

        # Define yourself dataset path
        self.output_dir = "./YOLOX_outputs2"
        self.data_dir = '/mnt/raid/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/Detector/yolox/autoriaNumberplateDataset-2021-12-12'
        self.train_ann = "yolox_train.json"
        self.val_ann = "yolox_val.json"

        self.num_classes = 1

        self.max_epoch = 1000
        self.data_num_workers = 32
        self.eval_interval = 1
        
    def get_data_loader(
        self, batch_size, is_distributed, no_aug=False, cache_img=False
    ):
        from yolox.data import (
            COCODataset,
            TrainTransform,
            YoloBatchSampler,
            DataLoader,
            InfiniteSampler,
            MosaicDetection,
            worker_init_reset_seed,
        )
        from yolox.utils import (
            wait_for_the_master,
            get_local_rank,
        )

        local_rank = get_local_rank()
        
        print("self.train_ann", self.train_ann)
        with wait_for_the_master(local_rank):
            dataset = COCODataset(
                data_dir=self.data_dir,
                json_file=self.train_ann,
                img_size=self.input_size,
                name="train",
                preproc=TrainTransform(
                    max_labels=50,
                    flip_prob=self.flip_prob,
                    hsv_prob=self.hsv_prob),
                cache=cache_img,
            )
        dataset = MosaicDetection(
            dataset,
            mosaic=not no_aug,
            img_size=self.input_size,
            preproc=TrainTransform(
                max_labels=120,
                flip_prob=self.flip_prob,
                hsv_prob=self.hsv_prob),
            degrees=self.degrees,
            translate=self.translate,
            mosaic_scale=self.mosaic_scale,
            mixup_scale=self.mixup_scale,
            shear=self.shear,
            enable_mixup=self.enable_mixup,
            mosaic_prob=self.mosaic_prob,
            mixup_prob=self.mixup_prob,
        )

        self.dataset = dataset
        print("len(self.dataset)", len(self.dataset))

        if is_distributed:
            batch_size = batch_size // dist.get_world_size()

        sampler = InfiniteSampler(len(self.dataset), seed=self.seed if self.seed else 0)

        batch_sampler = YoloBatchSampler(
            sampler=sampler,
            batch_size=batch_size,
            drop_last=False,
            mosaic=not no_aug,
        )

        dataloader_kwargs = {"num_workers": self.data_num_workers, "pin_memory": True}
        dataloader_kwargs["batch_sampler"] = batch_sampler

        # Make sure each process has different random seed, especially for 'fork' method.
        # Check https://github.com/pytorch/pytorch/issues/63311 for more details.
        dataloader_kwargs["worker_init_fn"] = worker_init_reset_seed

        train_loader = DataLoader(self.dataset, **dataloader_kwargs)

        return train_loader
    
    def get_eval_loader(self, batch_size, is_distributed, testdev=False, legacy=False):
        from yolox.data import COCODataset, ValTransform
        
        
        print("self.train_ann", self.val_ann)
        valdataset = COCODataset(
            data_dir=self.data_dir,
            json_file=self.val_ann,
            name="val",
            img_size=self.test_size,
            preproc=ValTransform(legacy=legacy),
        )

        if is_distributed:
            batch_size = batch_size // dist.get_world_size()
            sampler = torch.utils.data.distributed.DistributedSampler(
                valdataset, shuffle=False
            )
        else:
            sampler = torch.utils.data.SequentialSampler(valdataset)

        dataloader_kwargs = {
            "num_workers": self.data_num_workers,
            "pin_memory": True,
            "sampler": sampler,
        }
        dataloader_kwargs["batch_size"] = batch_size
        val_loader = torch.utils.data.DataLoader(valdataset, **dataloader_kwargs)

        return val_loader

Overwriting /mnt/raid/var/www/nomeroff-net/data/configs/yolox_s.py


# Download coco weigth

# Train

In [29]:
! cd {repo_path}; \
  wget -nc https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_s.pth; \
  CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=1 PYTHONPATH=:{repo_path} python3.9 ./tools/train.py -f {NOMEROFF_NET_DIR}/data/configs/yolox_s.py -d 1 -b 16 --fp16 -o -c ./yolox_s.pth;

File ‘yolox_s.pth’ already there; not retrieving.

2022-04-01 12:38:12 | INFO     | yolox.core.trainer:129 - args: Namespace(experiment_name='yolox_s', name=None, dist_backend='nccl', dist_url=None, batch_size=16, devices=1, exp_file='/mnt/raid/var/www/nomeroff-net/data/configs/yolox_s.py', resume=False, ckpt='./yolox_s.pth', start_epoch=None, num_machines=1, machine_rank=0, fp16=True, cache=False, occupy=True, logger='tensorboard', opts=[])
2022-04-01 12:38:12 | INFO     | yolox.core.trainer:130 - exp value:
╒═══════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ keys              │ values                                                                                                                       │
╞═══════════════════╪══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ seed              │ None            

2022-04-01 12:38:12 | INFO     | yolox.core.trainer:135 - Model Summary: Params: 8.94M, Gflops: 26.64
2022-04-01 12:38:17 | INFO     | yolox.core.trainer:307 - loading checkpoint for fine tuning
2022-04-01 12:38:17 | WARNING  | yolox.utils.checkpoint:24 - Shape of head.cls_preds.0.weight in checkpoint is torch.Size([80, 128, 1, 1]), while shape of head.cls_preds.0.weight in model is torch.Size([1, 128, 1, 1]).
2022-04-01 12:38:17 | WARNING  | yolox.utils.checkpoint:24 - Shape of head.cls_preds.0.bias in checkpoint is torch.Size([80]), while shape of head.cls_preds.0.bias in model is torch.Size([1]).
2022-04-01 12:38:17 | WARNING  | yolox.utils.checkpoint:24 - Shape of head.cls_preds.1.weight in checkpoint is torch.Size([80, 128, 1, 1]), while shape of head.cls_preds.1.weight in model is torch.Size([1, 128, 1, 1]).
2022-04-01 12:38:17 | WARNING  | yolox.utils.checkpoint:24 - Shape of head.cls_preds.1.bias in checkpoint is torch.Size([80]), while shape of head.cls_preds.1.bias in model i

2022-04-01 12:38:35 | INFO     | yolox.core.trainer:252 - epoch: 1/10, iter: 10/480, mem: 28562Mb, iter_time: 0.480s, data_time: 0.056s, total_loss: 24.2, iou_loss: 2.7, l1_loss: 1.4, conf_loss: 15.6, cls_loss: 4.5, lr: 4.340e-08, size: 640, ETA: 0:38:17
2022-04-01 12:38:39 | INFO     | yolox.core.trainer:252 - epoch: 1/10, iter: 20/480, mem: 28562Mb, iter_time: 0.469s, data_time: 0.003s, total_loss: 18.9, iou_loss: 2.3, l1_loss: 1.1, conf_loss: 11.9, cls_loss: 3.5, lr: 1.736e-07, size: 736, ETA: 0:37:48
2022-04-01 12:38:43 | INFO     | yolox.core.trainer:252 - epoch: 1/10, iter: 30/480, mem: 28562Mb, iter_time: 0.363s, data_time: 0.002s, total_loss: 21.5, iou_loss: 2.7, l1_loss: 1.2, conf_loss: 13.2, cls_loss: 4.4, lr: 3.906e-07, size: 544, ETA: 0:34:46
2022-04-01 12:38:49 | INFO     | yolox.core.trainer:252 - epoch: 1/10, iter: 40/480, mem: 28562Mb, iter_time: 0.627s, data_time: 0.001s, total_loss: 19.0, iou_loss: 2.4, l1_loss: 1.0, conf_loss: 12.1, cls_loss: 3.5, lr: 6.944e-07, size

2022-04-01 12:40:17 | INFO     | yolox.core.trainer:252 - epoch: 1/10, iter: 290/480, mem: 28562Mb, iter_time: 0.493s, data_time: 0.005s, total_loss: 2.4, iou_loss: 1.1, l1_loss: 0.3, conf_loss: 0.7, cls_loss: 0.4, lr: 3.650e-05, size: 768, ETA: 0:27:36
2022-04-01 12:40:26 | INFO     | yolox.core.trainer:252 - epoch: 1/10, iter: 300/480, mem: 28562Mb, iter_time: 0.937s, data_time: 0.006s, total_loss: 2.4, iou_loss: 1.2, l1_loss: 0.3, conf_loss: 0.5, cls_loss: 0.4, lr: 3.906e-05, size: 576, ETA: 0:28:58
2022-04-01 12:40:32 | INFO     | yolox.core.trainer:252 - epoch: 1/10, iter: 310/480, mem: 28562Mb, iter_time: 0.558s, data_time: 0.013s, total_loss: 2.4, iou_loss: 1.1, l1_loss: 0.3, conf_loss: 0.6, cls_loss: 0.4, lr: 4.171e-05, size: 800, ETA: 0:29:19
2022-04-01 12:40:34 | INFO     | yolox.core.trainer:252 - epoch: 1/10, iter: 320/480, mem: 28562Mb, iter_time: 0.210s, data_time: 0.002s, total_loss: 2.2, iou_loss: 1.0, l1_loss: 0.3, conf_loss: 0.5, cls_loss: 0.4, lr: 4.444e-05, size: 64

2022-04-01 12:41:27 | INFO     | yolox.core.trainer:351 - Save weights to ./YOLOX_outputs/yolox_s
2022-04-01 12:41:28 | INFO     | yolox.core.trainer:202 - ---> start train epoch2
2022-04-01 12:41:28 | INFO     | yolox.core.trainer:205 - --->No mosaic aug now!
2022-04-01 12:41:28 | INFO     | yolox.core.trainer:207 - --->Add additional L1 loss now!
2022-04-01 12:41:32 | INFO     | yolox.core.trainer:252 - epoch: 2/10, iter: 10/480, mem: 28562Mb, iter_time: 0.446s, data_time: 0.024s, total_loss: 2.4, iou_loss: 1.2, l1_loss: 0.3, conf_loss: 0.6, cls_loss: 0.4, lr: 1.042e-04, size: 576, ETA: 0:24:53
2022-04-01 12:41:38 | INFO     | yolox.core.trainer:252 - epoch: 2/10, iter: 20/480, mem: 28562Mb, iter_time: 0.522s, data_time: 0.010s, total_loss: 2.1, iou_loss: 1.1, l1_loss: 0.3, conf_loss: 0.4, cls_loss: 0.4, lr: 1.085e-04, size: 768, ETA: 0:25:04
2022-04-01 12:41:42 | INFO     | yolox.core.trainer:252 - epoch: 2/10, iter: 30/480, mem: 28562Mb, iter_time: 0.415s, data_time: 0.011s, total_

...

2022-04-01 12:57:12 | INFO     | yolox.core.trainer:252 - epoch: 9/10, iter: 60/480, mem: 28562Mb, iter_time: 0.171s, data_time: 0.001s, total_loss: 1.8, iou_loss: 1.0, l1_loss: 0.2, conf_loss: 0.3, cls_loss: 0.3, lr: 1.250e-04, size: 544, ETA: 0:04:02
2022-04-01 12:57:14 | INFO     | yolox.core.trainer:252 - epoch: 9/10, iter: 70/480, mem: 28562Mb, iter_time: 0.233s, data_time: 0.001s, total_loss: 1.3, iou_loss: 0.7, l1_loss: 0.2, conf_loss: 0.2, cls_loss: 0.3, lr: 1.250e-04, size: 768, ETA: 0:03:59
2022-04-01 12:57:17 | INFO     | yolox.core.trainer:252 - epoch: 9/10, iter: 80/480, mem: 28562Mb, iter_time: 0.239s, data_time: 0.001s, total_loss: 1.6, iou_loss: 0.8, l1_loss: 0.2, conf_loss: 0.3, cls_loss: 0.3, lr: 1.250e-04, size: 768, ETA: 0:03:56
2022-04-01 12:57:18 | INFO     | yolox.core.trainer:252 - epoch: 9/10, iter: 90/480, mem: 28562Mb, iter_time: 0.174s, data_time: 0.002s, total_loss: 1.8, iou_loss: 1.0, l1_loss: 0.2, conf_loss: 0.3, cls_loss: 0.3, lr: 1.250e-04, size: 640, E

2022-04-01 12:58:16 | INFO     | yolox.core.trainer:252 - epoch: 9/10, iter: 340/480, mem: 28562Mb, iter_time: 0.241s, data_time: 0.001s, total_loss: 1.5, iou_loss: 0.8, l1_loss: 0.2, conf_loss: 0.2, cls_loss: 0.3, lr: 1.250e-04, size: 768, ETA: 0:02:45
2022-04-01 12:58:18 | INFO     | yolox.core.trainer:252 - epoch: 9/10, iter: 350/480, mem: 28562Mb, iter_time: 0.169s, data_time: 0.001s, total_loss: 1.9, iou_loss: 1.0, l1_loss: 0.2, conf_loss: 0.4, cls_loss: 0.3, lr: 1.250e-04, size: 512, ETA: 0:02:42
2022-04-01 12:58:20 | INFO     | yolox.core.trainer:252 - epoch: 9/10, iter: 360/480, mem: 28562Mb, iter_time: 0.201s, data_time: 0.001s, total_loss: 1.5, iou_loss: 0.8, l1_loss: 0.2, conf_loss: 0.3, cls_loss: 0.3, lr: 1.250e-04, size: 736, ETA: 0:02:39
2022-04-01 12:58:22 | INFO     | yolox.core.trainer:252 - epoch: 9/10, iter: 370/480, mem: 28562Mb, iter_time: 0.181s, data_time: 0.001s, total_loss: 1.7, iou_loss: 0.9, l1_loss: 0.2, conf_loss: 0.3, cls_loss: 0.3, lr: 1.250e-04, size: 60

2022-04-01 12:59:01 | INFO     | yolox.core.trainer:252 - epoch: 10/10, iter: 20/480, mem: 28562Mb, iter_time: 0.170s, data_time: 0.001s, total_loss: 1.7, iou_loss: 0.9, l1_loss: 0.2, conf_loss: 0.2, cls_loss: 0.3, lr: 1.250e-04, size: 480, ETA: 0:02:02
2022-04-01 12:59:03 | INFO     | yolox.core.trainer:252 - epoch: 10/10, iter: 30/480, mem: 28562Mb, iter_time: 0.190s, data_time: 0.001s, total_loss: 1.6, iou_loss: 0.9, l1_loss: 0.2, conf_loss: 0.2, cls_loss: 0.3, lr: 1.250e-04, size: 640, ETA: 0:01:59
2022-04-01 12:59:05 | INFO     | yolox.core.trainer:252 - epoch: 10/10, iter: 40/480, mem: 28562Mb, iter_time: 0.169s, data_time: 0.001s, total_loss: 1.6, iou_loss: 0.9, l1_loss: 0.2, conf_loss: 0.3, cls_loss: 0.3, lr: 1.250e-04, size: 512, ETA: 0:01:56
2022-04-01 12:59:07 | INFO     | yolox.core.trainer:252 - epoch: 10/10, iter: 50/480, mem: 28562Mb, iter_time: 0.240s, data_time: 0.001s, total_loss: 1.9, iou_loss: 1.0, l1_loss: 0.2, conf_loss: 0.4, cls_loss: 0.3, lr: 1.250e-04, size: 80

2022-04-01 13:00:08 | INFO     | yolox.core.trainer:252 - epoch: 10/10, iter: 300/480, mem: 28562Mb, iter_time: 0.243s, data_time: 0.001s, total_loss: 1.8, iou_loss: 1.0, l1_loss: 0.2, conf_loss: 0.3, cls_loss: 0.3, lr: 1.250e-04, size: 768, ETA: 0:00:47
2022-04-01 13:00:10 | INFO     | yolox.core.trainer:252 - epoch: 10/10, iter: 310/480, mem: 28562Mb, iter_time: 0.198s, data_time: 0.001s, total_loss: 1.6, iou_loss: 0.8, l1_loss: 0.2, conf_loss: 0.3, cls_loss: 0.3, lr: 1.250e-04, size: 704, ETA: 0:00:44
2022-04-01 13:00:11 | INFO     | yolox.core.trainer:252 - epoch: 10/10, iter: 320/480, mem: 28562Mb, iter_time: 0.171s, data_time: 0.001s, total_loss: 1.7, iou_loss: 0.9, l1_loss: 0.2, conf_loss: 0.3, cls_loss: 0.3, lr: 1.250e-04, size: 512, ETA: 0:00:42
2022-04-01 13:00:13 | INFO     | yolox.core.trainer:252 - epoch: 10/10, iter: 330/480, mem: 28562Mb, iter_time: 0.169s, data_time: 0.001s, total_loss: 1.8, iou_loss: 0.9, l1_loss: 0.2, conf_loss: 0.4, cls_loss: 0.3, lr: 1.250e-04, size

2022-04-01 13:00:53 | INFO     | yolox.core.trainer:351 - Save weights to ./YOLOX_outputs/yolox_s
2022-04-01 13:00:53 | INFO     | yolox.core.trainer:194 - Training of experiment is done and the best AP is 77.60


In [31]:
!echo CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=1 PYTHONPATH=:{repo_path} python3.9 ./tools/train.py -f {NOMEROFF_NET_DIR}/data/configs/yolox_s.py -d 1 -b 16 --fp16 -o -c ./yolox_s.pth;

CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=1 PYTHONPATH=:/mnt/raid/var/www/nomeroff-net/nomeroff_net/tools/../../data/./repos/Detector/yolox python3.9 ./tools/train.py -f /mnt/raid/var/www/nomeroff-net/data/configs/yolox_s.py -d 1 -b 16 --fp16 -o -c ./yolox_s.pth
